## Import dependencies

In [16]:
import gurobipy as gp
from gurobipy import GRB

## Prepare data

In [ ]:

# --- 1. Data Preparation ---
subjects = ["Anatomie", "Biologie", "Chirurgie", "Diagnostic", "Epidemiologie", "Forensic", "Genetique"]
weights = {"Anatomie": 8, "Biologie": 7, "Chirurgie": 7, "Diagnostic": 6, "Epidemiologie": 6, "Forensic": 5, "Genetique": 6}

# Candidate grades (Xavier vs Yvonne)
grades_x = {"Anatomie": 85, "Biologie": 81, "Chirurgie": 71, "Diagnostic": 69, "Epidemiologie": 75, "Forensic": 81, "Genetique": 88}
grades_y = {"Anatomie": 81, "Biologie": 81, "Chirurgie": 75, "Diagnostic": 63, "Epidemiologie": 67, "Forensic": 88, "Genetique": 95}
grades = {
    'x': {"A": 85, "B": 81, "C": 71, "D": 69, "E": 75, "F": 81, "G": 88},
    'y': {"A": 81, "B": 81, "C": 75, "D": 63, "E": 67, "F": 88, "G": 95},
    'z': {"A": 74, "B": 89, "C": 74, "D": 81, "E": 68, "F": 84, "G": 79},
    't': {"A": 74, "B": 71, "C": 84, "D": 91, "E": 77, "F": 76, "G": 73},
    'u': {"A": 72, "B": 66, "C": 75, "D": 85, "E": 88, "F": 66, "G": 93},
    'v': {"A": 71, "B": 73, "C": 63, "D": 92, "E": 76, "F": 79, "G": 93},
    'w': {"A": 79, "B": 69, "C": 78, "D": 76, "E": 67, "F": 84, "G": 79},
    'w_prime': {"A": 57, "B": 76, "C": 81, "D": 76, "E": 82, "F": 86, "G": 77},
}

## Compute pros, cons and deltas

In [18]:
def compute_pros_cons_deltas(grades_x, grades_y, subjects = subjects, weights = weights):
    # Calculate Contributions (deltas)
    deltas = {}
    pros = []
    cons = []

    for s in subjects:
        # Contribution = weight * (grade_x - grade_y)
        diff = grades_x[s] - grades_y[s]
        contrib = weights[s] * diff
        deltas[s] = contrib
        
        if contrib > 0:
            pros.append(s)
        elif contrib < 0:
            cons.append(s)


    return pros, cons, deltas

In [19]:
pros, cons, deltas = compute_pros_cons_deltas(grades_x, grades_y)

print(f"\nPros: {pros}")
print(f"Cons: {cons}")


Pros: ['Anatomie', 'Diagnostic', 'Epidemiologie']
Cons: ['Chirurgie', 'Forensic', 'Genetique']


## Enumerate all 1-1 trade-offs

In [20]:
for pro in pros:
    for con in cons:
        if weights[pro]*grades_x[pro] - weights[con]*grades_x[con] > 0:
            print(f"Trade-off: Improve {con} by sacrificing {pro}")

Trade-off: Improve Chirurgie by sacrificing Anatomie
Trade-off: Improve Forensic by sacrificing Anatomie
Trade-off: Improve Genetique by sacrificing Anatomie
Trade-off: Improve Forensic by sacrificing Diagnostic
Trade-off: Improve Forensic by sacrificing Epidemiologie


## Create Gurobi Model

In [21]:
def explanation_1_1(pros, cons, deltas):
    # Create a new model
    m = gp.Model("Explanation_1_1")

    # Decision Variables: x[p, c] = 1 if pro p explains con c
    x = {}

    # Only create variables for VALID trade-offs (where delta_p + delta_c >= 0)
    # This implicitly handles the "strength" constraint
    for p in pros:
        for c in cons:
            if deltas[p] + deltas[c] >= 0:
                x[p, c] = m.addVar(vtype=GRB.BINARY, name=f"match_{p}_{c}")

    # Update model to integrate variables
    m.update()

    # Constraint 1: Every CON must be covered exactly once
    for c in cons:
        m.addConstr(gp.quicksum(x[p, c] for p in pros if (p, c) in x) == 1, name=f"cover_{c}")

    # Constraint 2: Every PRO can be used at most once
    for p in pros:
        m.addConstr(gp.quicksum(x[p, c] for c in cons if (p, c) in x) <= 1, name=f"use_once_{p}")

    # Objective: Just find a feasible solution. 
    # (Gurobi will try to satisfy constraints. If impossible, it returns Infeasible)
    m.setObjective(0, GRB.MINIMIZE)

    # Optimize
    m.optimize()

In [22]:
explanation_1_1(pros, cons, deltas)

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Academic license 2755059 - for non-commercial use only - registered to ma___@student-cs.fr
Optimize a model with 6 rows, 6 columns and 12 nonzeros (Min)
Model fingerprint: 0x0ba5136a
Model has 0 linear objective coefficients
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 14 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


## Print out results

In [23]:
# --- 3. Output Results ---
if m.status == GRB.OPTIMAL:
    print("\n--- Explanation Found (Type 1-1) ---")
    for p in pros:
        for c in cons:
            if (p, c) in x and x[p, c].X > 0.5:
                print(f"Trade-off: Because {p} (+{deltas[p]}) compensates for {c} ({deltas[c]})")
elif m.status == GRB.INFEASIBLE:
    print("\nNo (1-1) explanation exists for this comparison.")
    # Optional: Calculate IIS to see which constraints failed
    m.computeIIS()
    m.write("model.ilp")
    print("Certificate of non-existence written to model.ilp")


--- Explanation Found (Type 1-1) ---
Trade-off: Because Anatomie (+32) compensates for Chirurgie (-28)
Trade-off: Because Diagnostic (+36) compensates for Forensic (-35)
Trade-off: Because Epidemiologie (+48) compensates for Genetique (-42)


## Show limits of current model